In [2]:
import os
import re
import glob
import uproot
import numpy  as np
import pandas as pd

from os.path import expandvars, join, basename

import matplotlib.pyplot as plt
%matplotlib widget

In [3]:
def read_time_window_data(filename):

    columns = ["Event", "ntws", "cluster", "t0", "t1", "pt", "px", "py", "pz", "nsub", "t", "G"]
    df = pd.DataFrame(columns=columns)
    
    f = uproot.open(filename)
    t = f["fiTQun"]
    fqntwnd          = t["fqntwnd"]         .array().to_numpy()-1
    fqtwnd_iclstr    = t["fqtwnd_iclstr"]   .array().to_numpy()
    fqtwnd_npeak     = t["fqtwnd_npeak"]    .array().to_numpy()
    fqtwnd_prftt0    = t["fqtwnd_prftt0"]   .array().to_numpy()
    fqtwnd_prftpos   = t["fqtwnd_prftpos"]  .array().to_numpy()
    fqtwnd           = t["fqtwnd"]          .array().to_numpy()
    fqtwnd_peakt0    = t["fqtwnd_peakt0"]   .array().to_numpy()
    fqtwnd_peakiness = t["fqtwnd_peakiness"].array().to_numpy()

    nevents = len(fqntwnd)

    for ev, event in enumerate(range(nevents)):
        nc = fqntwnd[ev]
        for cluster in fqtwnd_iclstr[ev]:
            ts   = fqtwnd        [ev, cluster]
            pt   = fqtwnd_prftt0 [ev, cluster]
            ppos = fqtwnd_prftpos[ev, cluster]
            nsub = fqtwnd_npeak  [ev, cluster]
            for subp in range(nsub):
                t = fqtwnd_peakt0   [ev, cluster, subp]
                G = fqtwnd_peakiness[ev, cluster, subp]
                df.loc[len(df)] = (event, nc, cluster, *ts, pt, *ppos, nsub, t, G)
    f.close()
    return df


def read_subevent_data(filename):

    columns = ["Event", "nsub", "tw", "peak", "N", "Q", "Q0R", "nll0R", "n50", "q50"]
    df = pd.DataFrame(columns=columns)
    
    f = uproot.open(filename)
    t = f["fiTQun"]

    fqnse     = t["fqnse"]    .array().to_numpy()
    fqitwnd   = t["fqitwnd"]  .array().to_numpy()
    fqipeak   = t["fqipeak"]  .array().to_numpy()
    fqnhitpmt = t["fqnhitpmt"].array().to_numpy()
    fqtotq    = t["fqtotq"]   .array().to_numpy()
    fq0rtotmu = t["fq0rtotmu"].array().to_numpy()
    fq0rnll   = t["fq0rnll"]  .array().to_numpy()
    fqn50     = t["fqn50"]    .array().to_numpy()
    fqq50     = t["fqq50"]    .array().to_numpy()

    nevents = len(fqnse)

    for ev, event in enumerate(range(nevents)):
        nsub = fqnse[ev]
        for subp in range(nsub):
            tw    = fqitwnd  [ev, subp]
            peak  = fqipeak  [ev, subp]
            N     = fqnhitpmt[ev, subp]
            Q     = fqtotq   [ev, subp]
            Q0R   = fq0rtotmu[ev, subp]
            nll0R = fq0rnll  [ev, subp]
            n50   = fqn50    [ev, subp]
            q50   = fqq50    [ev, subp]
            df.loc[len(df)] = (event, nsub, tw, peak, N, Q, Q0R, nll0R, n50, q50)
    f.close()
    return df


def read_1Ring_data(filename, pids, event_counter=0):

    columns = ["Event", "peak", "pid", "pc", "p", "t", "x", "y", "z", "theta", "phi", "Q1R", "nll1R", "L", "Eloss"]
    df = pd.DataFrame(columns=columns)
    
    f = uproot.open(filename)
    t = f["fiTQun"]

    fqnse   = t["fqnse"]  .array()
    fqipeak = t["fqipeak"].array()
    nevents = len(fqnse)

    fq1rpcflg = t["fq1rpcflg"].array()
    fq1rtotmu = t["fq1rtotmu"].array()
    fq1rnll   = t["fq1rnll"]  .array()
    fq1rmom   = t["fq1rmom"]  .array()
    fq1rt0    = t["fq1rt0"]   .array()
    fq1rpos   = t["fq1rpos"]  .array()
    fq1rdir   = t["fq1rdir"]  .array()
    fq1rdconv = t["fq1rdconv"].array()
    fq1reloss = t["fq1reloss"].array()

    for ev in range(nevents):
        event = event_counter + ev
        nsub = fqnse[ev]
        for subp in range(nsub):
            peak = fqipeak [ev, subp]
            for pid in pids:
                pc    = fq1rpcflg[ev, subp, pid]
                mom   = fq1rmom  [ev, subp, pid]
                t     = fq1rt0   [ev, subp, pid]
                pos   = fq1rpos  [ev, subp, pid]
                dir   = fq1rdir  [ev, subp, pid]
                Q1R   = fq1rtotmu[ev, subp, pid]
                nll1R = fq1rnll  [ev, subp, pid]
                L     = fq1rdconv[ev, subp, pid]
                Eloss = fq1reloss[ev, subp, pid]

                theta = np.arccos(dir[2])
                phi   = np.arctan2(dir[1], dir[0])
                if (np.sign(phi)<0): phi += 2.*np.pi

                df.loc[len(df)] = (event, peak, pid, pc, mom, t, *pos, theta, phi, Q1R, nll1R, L, Eloss)
    f.close()
    return df


In [20]:
filename = expandvars("$LUSTRE/Validation/mu-/Ryo/out/fq_mu-_100_6.root")

In [28]:
f = uproot.open(filename)
t = f["fiTQun"]

fqnse   = t["fqnse"]  .array()
fqipeak = t["fqipeak"].array()
nevents = len(fqnse)

fq1rpcflg = t["fq1rpcflg"].array()
fq1rtotmu = t["fq1rtotmu"].array()
fq1rnll   = t["fq1rnll"]  .array()
fq1rmom   = t["fq1rmom"]  .array()
fq1rt0    = t["fq1rt0"]   .array()
fq1rpos   = t["fq1rpos"]  .array()
fq1rdir   = t["fq1rdir"]  .array()
fq1rdconv = t["fq1rdconv"].array()
fq1reloss = t["fq1reloss"].array()

fqitwnd = t["fqitwnd"].array()

In [47]:
fqntwnd        = t["fqntwnd"]       .array()
fqtwnd_iclstr  = t["fqtwnd_iclstr"] .array()
fqtwnd_npeak   = t["fqtwnd_npeak"]  .array()
fqtwnd_prftt0  = t["fqtwnd_prftt0"] .array()
fqtwnd_prftpos = t["fqtwnd_prftpos"].array()
fqtwnd         = t["fqtwnd"]        .array()
fqtwnd_peakt0  = t["fqtwnd_peakt0"] .array()

fqtwnd_peakiness = t["fqtwnd_peakiness"].array()

In [48]:
fqtwnd_peakiness

<Array [[[78.6, 0, 0, ..., 0, 0, 0], ...], ...] type='50 * var * 10 * float32'>

In [ ]:
for ev in range(nevents):
        event = event_counter + ev
        nsub = fqnse[ev]
        for subp in range(nsub):
            peak = fqipeak [ev, subp]

In [11]:
def read_1Ring_data(filename, pids, event_counter=0):

    columns = ["Event", "peak", "pid", "pc", "p", "t", "x", "y", "z", "theta", "phi", "Q1R", "nll1R", "L", "Eloss"]
    df = pd.DataFrame(columns=columns)
    
    f = uproot.open(filename)
    t = f["fiTQun"]

    fqnse   = t["fqnse"]  .array()
    fqipeak = t["fqipeak"].array()
    nevents = len(fqnse)

    fq1rpcflg = t["fq1rpcflg"].array()
    fq1rtotmu = t["fq1rtotmu"].array()
    fq1rnll   = t["fq1rnll"]  .array()
    fq1rmom   = t["fq1rmom"]  .array()
    fq1rt0    = t["fq1rt0"]   .array()
    fq1rpos   = t["fq1rpos"]  .array()
    fq1rdir   = t["fq1rdir"]  .array()
    fq1rdconv = t["fq1rdconv"].array()
    fq1reloss = t["fq1reloss"].array()

    for ev in range(nevents):
        event = event_counter + ev
        nsub = fqnse[ev]
        for subp in range(nsub):
            peak = fqipeak [ev, subp]
            for pid in pids:
                pc    = fq1rpcflg[ev, subp, pid]
                mom   = fq1rmom  [ev, subp, pid]
                t     = fq1rt0   [ev, subp, pid]
                pos   = fq1rpos  [ev, subp, pid]
                dir   = fq1rdir  [ev, subp, pid]
                Q1R   = fq1rtotmu[ev, subp, pid]
                nll1R = fq1rnll  [ev, subp, pid]
                L     = fq1rdconv[ev, subp, pid]
                Eloss = fq1reloss[ev, subp, pid]

                theta = np.arccos(dir[2])
                phi   = np.arctan2(dir[1], dir[0])
                if (np.sign(phi)<0): phi += 2.*np.pi

                df.loc[len(df)] = (event, peak, pid, pc, mom, t, *pos, theta, phi, Q1R, nll1R, L, Eloss)
    f.close()
    return df

,Event,peak,pid,pc,p,t,x,y,z,theta,phi,Q1R,nll1R,L,Eloss
0,0.0,0.0,1.0,0.0,17.906454,947.520813,6.169152,-0.261485,-64.619545,3.071930,3.892226,146.033752,508.255157,0.0,0.0
1,0.0,0.0,1.0,0.0,34.290550,2376.218750,-3.385581,-1.461992,-37.186642,2.861258,1.559413,277.204346,853.829895,0.0,0.0
2,1.0,0.0,1.0,0.0,19.545904,947.395752,-3.872675,0.845973,-65.820801,3.031145,5.047125,160.421753,522.307556,0.0,0.0
3,1.0,0.0,1.0,0.0,40.985374,2772.034180,-1.201308,8.906101,-34.394985,0.724089,4.899035,354.498871,1472.732300,0.0,0.0
4,2.0,0.0,1.0,0.0,19.223577,947.678833,-5.607340,-7.834820,-70.555191,3.006122,1.906012,159.657318,540.492859,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,47.0,0.0,1.0,0.0,8.337298,3114.691162,3.313491,-40.697620,-51.454029,1.910519,5.243624,61.478642,444.730103,0.0,0.0
91,48.0,0.0,1.0,0.0,15.830165,943.959045,4.216865,5.119912,19.003263,2.698192,1.297752,131.496368,763.143982,0.0,0.0
92,48.0,1.0,1.0,0.0,67.834221,1619.449951,4.473916,8.944179,-26.595804,1.116775,1.839181,518.707397,1475.204102,0.0,0.0
93,49.0,0.0,1.0,0.0,21.467506,947.654114,0.404083,-1.554938,-65.778030,3.047922,3.620254,175.175705,603.594910,0.0,0.0
